In [ ]:
# imports packages
from __future__ import print_function 
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
import pandas as pd
#import csv
import random
import scipy.integrate as integrate
from numpy import genfromtxt
from random import randrange
from numpy.polynomial import Polynomial as P
from IPython.display import Latex
import time
from decimal import Decimal

# DM profile for a mass bin

## Create radial profile of a field

In [14]:
def radial_profile(data, center_x,center_y):
    y, x = np.indices((data.shape))
    r = np.sqrt((x - center_x)**2 + (y - center_y)**2)
    r = r.astype(np.int)

    tbin = np.bincount(r.ravel(), data.ravel())
    nr = np.bincount(r.ravel())
    radialprofile = tbin / nr
    return radialprofile 

## Calculate Dispersion measure for each mass bin

In [4]:
def DM_for_mass_bin(halo_array,df,crop_grid):
    
    res=halo_array.shape[0]
    trim_dim=crop_grid
#     DM_rad = np.zeros([len(df), 2*trim_dim+1,2*trim_dim+1])
    
    DM_rad = np.zeros([len(df), trim_dim,trim_dim])
    
#     ix = (df['ix'].values)
#     iy = (df['iy'].values)
    
    
    ix = ((np.around((4*res/1024)*((df['x'].values)/(250/256))))%(res)).astype(int)
    iy = ((np.around((4*res/1024)*((df['y'].values)/(250/256))))%(res)).astype(int)
#     print(ix)
    
    
        
    
    for i in range(0,len(df)):            
            
        if ix[i] > int(trim_dim) and ix[i] < res - int(trim_dim) and iy[i] > int(trim_dim) and iy[i] < res- int(trim_dim):
#             trimmed = halo_array[ix[i]-int(trim_dim):ix[i]+int(trim_dim+1),iy[i]-int(trim_dim):iy[i]+int(trim_dim +1)]
            trimmed = halo_array[ix[i]-int(trim_dim/2):ix[i]+int(trim_dim/2),iy[i]-int(trim_dim/2):iy[i]+int(trim_dim/2)]
            DM_rad[i,:,:] = trimmed
    

    return DM_rad
        
        

In [28]:
# Function: calculates the radial profile for each mass bin

def radial_profile_array(DM_halo_array,halos_per_mass_bin,len_rad_ar):
    
    trim_dim=DM_halo_array.shape[1]
    center=int(trim_dim/2)
    
    DM_mass_bin= np.zeros([len(halos_per_mass_bin),trim_dim,trim_dim])
    rad_prof_mass_bin = np.zeros([len(halos_per_mass_bin),len_rad_ar])

    
    # The -1 has to be thought more about
    for i in range(0,len(halos_per_mass_bin)-1):
        DM_mass_bin[i,:,:]= np.mean(DM_halo_array[halos_per_mass_bin[i]:halos_per_mass_bin[i+1],:,:],axis=0)
        rad_prof_mass_bin[i,:] = radial_profile(DM_mass_bin[i,:,:],center,center)
    
    return rad_prof_mass_bin,DM_mass_bin
    
    
    
    

In [29]:
# Function: plots radial profiles for different halo mass bins

def plot_radial_profile(radial_profile_array,mass_array,radial_extent,num_of_plots):
    
    plt.figure(figsize=(10,10))
    plot_x_dim = radial_profile_array.shape[1]
    
    for i in range(0,len(mass_array)-1,num_of_plots):
        
        plt.semilogx(np.linspace(0,radial_extent,plot_x_dim),radial_profile_array[i,:]-mean_DM, label= 'Mass = %.1E' % Decimal(mass_array[i]))
        plt.legend(loc='upper right')
        plt.xlabel('Radial distance (Mpc)')
        plt.ylabel('DM - <DM>')
    plt.savefig('DMvsRad_ImpactParam')

In [30]:
# Function: plots radial profiles for different halo mass bins

def plot_radial_profile_2(radial_profile_array_1,radial_profile_array_2,mass_array,radial_extent,num_of_plots):
    
    plt.figure(figsize=(10,10))
    plot_x_dim_1 = radial_profile_array_1.shape[1]
    print(plot_x_dim_1)
    plot_x_dim_2 = radial_profile_array_2.shape[1]
    print(plot_x_dim_2)
    
    for i in range(0,len(mass_array)-1,num_of_plots):
        
        plt.semilogx(np.linspace(0,radial_extent,plot_x_dim_1),radial_profile_array_1[i,:]-mean_DM, label= 'Mass = %.1E' % Decimal(mass_array[i]))
        plt.semilogx(np.linspace(0,radial_extent,plot_x_dim_2),radial_profile_array_2[i,:], linestyle='--',label= 'Mass = %.1E' % Decimal(mass_array[i]))
        plt.legend(loc='upper right')
        plt.xlabel('Radial distance (Mpc)')
        plt.ylabel('DM - <DM>')
        plt.ylim(0,250)
    plt.savefig('DMvsRad_ImpactParam')

In [31]:
def plot_radial_profile_3(radial_profile_array_1,radial_profile_array_2,radial_profile_array_3,mass_array,radial_extent,num_of_plots):
    
    plt.figure(figsize=(10,10))
    plot_x_dim_1 = radial_profile_array_1.shape[1]
    plot_x_dim_2 = radial_profile_array_2.shape[1]
    plot_x_dim_3 = radial_profile_array_3.shape[1]
    
    for i in range(0,len(mass_array)-1,num_of_plots):
        
        plt.semilogx(np.linspace(0,radial_extent,plot_x_dim_1),radial_profile_array_1[i,:]-75, label= 'Mass = %.1E' % Decimal(mass_array[i]))
        plt.semilogx(np.linspace(0,radial_extent,plot_x_dim_2),radial_profile_array_2[i,:]-75, linestyle='--',label= 'Mass = %.1E' % Decimal(mass_array[i]))
        plt.semilogx(np.linspace(0,radial_extent,plot_x_dim_3),radial_profile_array_3[i,:]-75, linestyle='-.',label= 'Mass = %.1E' % Decimal(mass_array[i]))
        plt.legend(loc='upper right')
        plt.xlabel('Radial distance (Mpc)')
        plt.ylabel('DM - <DM>')
        plt.ylim(0,300)
    plt.savefig('DMvsRad_ImpactParam')

## Single function for radial profile of DM for a given DM array and grid size

In [33]:
# Function

def DM_vs_radius(DM_array,halo_data_frame,crop_grid_dim, mass_bins, hist_mass_bins):
    
#     len_rad_ar = radial_profile(DM_for_mass_bin(DM_array,halo_data_frame,crop_grid_dim)[0,:,:],int(crop_grid_dim),int(crop_grid_dim)).shape[0]
    
    # I can run profile of masks function to get this instead of running the above function
    len_rad_ar=15
    
    trimmed_ar = DM_for_mass_bin(DM_array,halo_data_frame,crop_grid_dim)
#     print((trimmed_ar.shape)[-1])
    
    final_ar=radial_profile(trimmed_ar[0,:,:],(trimmed_ar.shape)[-1]/2,(trimmed_ar.shape)[-1]/2)
        
#     return radial_profile_array(DM_for_mass_bin(DM_array,halo_data_frame,crop_grid_dim),hist_mass_bins,((final_ar.shape)[0]))

    output = radial_profile_array(trimmed_ar,hist_mass_bins,((final_ar.shape)[0]))
    
    return output[0],output[1],trimmed_ar
    

## Calculate profiles of masks 

In [10]:
def profile_of_masks(mask_array):
    
    mask_prof_ar_shape = mask_array.shape
    prof_num=mask_prof_ar_shape[0]
    mask_center=int(mask_prof_ar_shape[1]/2)
    
    dim_rad_ar=(radial_profile(mask_array[0,:,:], mask_center,mask_center)).shape[0]

    prof_masks = np.zeros([prof_num,dim_rad_ar])
    
    for i in range(0,prof_num):
    
        prof_masks[i,:] = radial_profile(mask_array[i,:,:], mask_center,mask_center)
        
    
    return prof_masks
    
    